In [ ]:
# SEND DATA TO GOOGLE FORM FUNCTION
import requests
from bs4 import BeautifulSoup
import json

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

# MATHS ABILITY TEST CODE
import time
import random
from IPython.display import display, HTML, clear_output
import pandas as pd

# Function that generates questions of different difficulty levels
def generate_question(difficulty):
    if difficulty == 1:
        num1 = random.randint(1, 10)
        num2 = random.randint(1, 10)
    elif difficulty == 2:
        num1 = random.randint(10, 20)
        num2 = random.randint(1, 10)
    else:
        num1 = random.randint(10, 20)
        num2 = random.randint(10, 20)
    
    operator = random.choice(['+', '-', '*'])
    question = f"{num1} {operator} {num2}"
    
    return question, eval(question)

# Function that displays each part of the question separately (for 1.3 seconds)
def display_question_parts(question):
    parts = question.split()
    for i, part in enumerate(parts):
        clear_output(wait=True)
        display(HTML(f"<p style='font-size: 30px; color: black;'>{part}</p>"))
        if i == 1:
            time.sleep(1.3) # flash each part of the question onscreen for 1.3 seconds
        else:
            time.sleep(1.3)
            clear_output(wait=True)
            time.sleep(0.3)  # brief pause before next part (0.3 seconds duration)

# Function for running the entire quiz, using the functions above
def run_quiz():
    random.seed(42)  # Setting a random seed for reproducibility

    id_instructions = """
    Enter your anonymised ID
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifer would be CBTC
    """
    print(id_instructions)
    user_id = input("> ")
    print("User entered id:", user_id)
    
    name = user_id
    age = input("Enter your age: ")

    results = {'username': username, 'age': age, 'score': 0, 'total_time': 0, 'avg_time': 0, 'raw_results': []}
    
    total_score = 0
    total_time = 0
    difficulty = 1
    question_duration = 1.3  # in seconds
    quiz_duration = 180  # in seconds
    
    start_time = time.time()
    
    while time.time() - start_time < quiz_duration:
        question, answer = generate_question(difficulty)
        
        clear_output(wait=True)
        display_question_parts(question)
        
        start_question_time = time.time()
        user_answer = input("Your answer: ")
        end_question_time = time.time()
        
        try:
            user_answer = float(user_answer)
        except ValueError:
            user_answer = None

        results['raw_results'].append(user_answer)  # Saving raw user inputs
        
        if user_answer == answer:
            total_score += 1
            total_time += end_question_time - start_question_time
        
        time.sleep(question_duration)
        
        difficulty += 1
    
    results['score'] = total_score
    results['total_questions'] = difficulty - 1
    if total_score > 0:
        results['avg_time'] = total_time / total_score
    else:
        results['avg_time'] = None
    results['total_time'] = total_time
    
    return results, name, age, total_score, results['avg_time']

quiz_results, name, age, score, avg_time = run_quiz()
print("Quiz results:", quiz_results)
print("Name:", name)
print("Age:", age)
print("Score:", score)
print("Average Time:", avg_time)

# data points from results that will be saved outside of the function & uploaded to form & used in data analysis
mydata = {
    'name': [],
    'age': [],
    'avg_time': [],
    'score': [],
    'raw_resuls':[],
    }
    
mydataframe = pd.DataFrame(mydata)
mydataframe
    
myjson = mydataframe.to_json()
myjson
    
# DATA TO UPLOAD TO GOOGLE FORM
answer_to_upload_dict = {
     "name": name,
     "age": age,
     "avg_time": avg_time,
     "score": score,
     }
    
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdLdtviU8xti1zlW1y6JnotvYPbNn2BfHIbasduu9gj0nFW7w/viewform?usp=sf_link"
send_to_google_form(answer_to_upload_dict,form_url)
    
# DISCLAIMER
print("")
print("Please read:")
print("")
print("We wish to record your response data")
print("in an anonymised public data repository. ")
print("Your data will be used for educational teaching purposes only.")
print("")
print("Please type yes in the box below if you consent to the upload.")
result = input("> ")
if result == "yes":
    print("Thanks - your data will be uploaded.")
    print("Thank you for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("if you have any questions or concerns")
    print("regarding the stored results.")
    send_to_google_form(answer_to_upload_dict, form_url)
else:
    print("No problem, your data will not be saved.")
    print("We hope you enjoyed the test!")